1. 문서의 내용을 읽는다
2. 문서를 쪼갠다    
    -토큰수 초과로 답변을 생성하지 못할수있꼬
    -문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3.임베딩 -> 벡터 데이터베이스에 저장
4.질문이 있으럗 벡터 데이터베이스에 유사도검색
5.유사도 검색으로 가져온 문서를 LLM에 질문과 같이전달


In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax.docx")
documents_list = loader.load_and_split(text_splitter = text_splitter)





In [2]:
len(documents_list)


193

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
 








In [14]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=documents_list, embedding=embedding, persist_directory="./chroma_db", collection_name="chroma-tax")


In [15]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)


In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")








In [17]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해주세요

[context]
{retrieved_docs}

[question]
{query} 
"""


In [18]:
ai_message = llm.invoke(prompt)


In [19]:
ai_message.content

'직장인의 소득세를 계산하기 위해서는 소득세율과 함께 다양한 공제 항목을 고려해야 합니다. 소득세는 종합소득세로 분류되며, 근로소득공제, 인적공제, 특별공제 등 여러 가지 공제를 적용한 후 산출세율을 적용하게 됩니다.\n\n기본적인 소득세 계산 절차는 다음과 같습니다:\n\n1. **총급여액**: 연봉 5,000만 원\n2. **근로소득공제**: 근로소득공제는 총급여액에 따라 다르게 적용되며, 일정 비율로 공제됩니다. \n   - 예: \\(5,000만원 \\times 20\\% + 200만원 = 1,200만원\\) (예시)\n3. **과세표준 계산**: \\(5,000만원 - 근로소득공제(1,200만원) = 3,800만원\\)\n4. **산출세액 계산**: 과세표준에 해당하는 소득세율을 적용합니다. 대한민국의 소득세율은 차등세율 체계로, 과세표준에 따라 6%~45%까지 적용됩니다.\n5. **세액공제 및 차감**: 기본세액공제나 근로소득세액공제 등을 적용해 최종 납부할 세액을 산출하게 됩니다.\n\n예를 들어, 3,800만 원에 해당하는 소득세율로 산출세액을 계산하고, 각종 세액공제를 적용한 후 최종 소득세를 결정하게 됩니다.\n\n구체적인 계산은 사용자의 상황에 따라 변동될 수 있으며, 신상정보나 각종 공제 항목이 필요하므로, 세무 전문가와 상담하는 것을 추천드립니다.'

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 몇 가지 중요한 점을 고려해야 합니다. 여기에는 기본적으로 과세표준, 세율, 소득공제 및 세액공제가 포함됩니다. 우선 기본적인 절차를 설명드리겠습니다.\n\n1. **과세표준 계산**:\n   - 연봉 5천만원에서 근로소득 공제를 차감합니다. 근로소득 공제는 일정 수식에 따라 계산됩니다.\n\n2. **세율 적용**:\n   - 과세표준에 따라 소득세율을 적용합니다. 한국의 소득세는 구간별 누진세율 구조를 따르고 있습니다. 각 구간에 따라 다른 세율이 적용됩니다.\n\n3. **세액공제 적용**:\n   - 기부금, 의료비, 교육비 등 다양한 세액공제 항목을 반영하여 계산된 세액에서 공제합니다.\n\n대략적인 계산을 예를 들어 설명드리자면 다음과 같습니다. 이것은 실제 공제 및 세율에 대한 계산의 예시일 뿐이며, 정확한 세금 계산은 세법에 따른 상세한 정보와 개인 상황에 따라 다를 수 있습니다.\n\n근로소득 공제 및 일반적인 세율을 대략적으로 적용하면:\n- 5천만원에서 근로소득 공제를 적용하여 과세표준을 산출합니다.\n- 과세표준에 따라 대략적인 소득세를 계산하고, 기본적인 세액공제를 적용하여 최종 소득세를 산출합니다.\n\n정확한 금액은 개인의 공제 항목과 적용 가능한 세액공제 등 다양한 사항에 따라 달라질 수 있습니다. 자세한 금액 산정은 국세청의 소득세 자동계산 프로그램을 이용하시거나 세무 전문가와 상담하시기를 권장드립니다.'

In [20]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")








d:\study\inflearn-llm\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
